In [0]:
# Módulos necesarios
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [0]:
%sql
select 
  *
from
  workspace.default.bronze_2025_2026
limit
  10;

In [0]:
# -----------------------------
# 1) Load Bronze
# -----------------------------
df_bronze = spark.table("workspace.default.bronze_2025_2026")

# -----------------------------
# 2) Crear df_silver con limpieza de descripcion
# -----------------------------
df_silver = (
    df_bronze
    .withColumnRenamed('descripcion_del_bien_ofertado', 'descripcion')
    .withColumn('descripcion', F.regexp_replace(F.col('descripcion'), '["\']', ''))
    .withColumn('descripcion', F.trim(F.col('descripcion')))
    .withColumn('descripcion', F.regexp_replace(F.col('descripcion'), r'\s+', ' '))
    .withColumn('minimo_2025', F.col('minimo_2025').cast('double'))
    .withColumn('maximo_2025', F.col('maximo_2025').cast('double'))
    .withColumn('minimo_2026', F.col('minimo_2026').cast('double'))
    .withColumn('maximo_2026', F.col('maximo_2026').cast('double'))
    .withColumn('totales_min', F.col('minimo_2025') + F.col('minimo_2026'))
    .withColumn('totales_max', F.col('maximo_2025') + F.col('maximo_2026'))
    .select(
        'no_partida',
        'clave',
        'descripcion',
        'minimo_2025',
        'maximo_2025',
        'minimo_2026',
        'maximo_2026',
        'totales_min',
        'totales_max'
    )
)

df_silver.limit(10).display()

In [0]:
# Guardamos la tabla Delta
table = "workspace.default.silver_2025_2026"
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(table)

print(f"✅ Tabla silver creada: {table}")